In [1]:
!python preprocess.py \
    --scrape-studies \
    --scrape-spreadsheet \
    --use-default-paths




Scraping studies...
100%|█████████████████████████████████████| 12259/12259 [25:30<00:00,  8.01it/s]
Fetched studies. Checking integrity...
Saving studies...
Raw studies saved.



Scraping spreadsheet data...
Verifying URLs...
100%|█████████████████████████████████████████| 488/488 [00:18<00:00, 26.17it/s]
Removing 398 invalid URLS
Scraping and extracting PDFs...
 63%|███████████████████████████▏               | 57/90 [00:31<00:28,  1.15it/s]Error opening PDF:  https://iucn-csg.org/wp-content/uploads/2010/03/Final-IUCN-Publication-Jan-7-2015-1.pdfError opening PDF:  
https://iucn-csg.org/wp-content/uploads/2010/03/Final-IUCN-Publication-Jan-7-2015-1.pdf
100%|███████████████████████████████████████████| 90/90 [01:23<00:00,  1.07it/s]
Scraped spreadsheet data. Saving...
Spreadsheet data saved.



Using default paths:
Irrelevant: ../../data/unprocessed/irrelevant/
Spreadsheet: ../../data/unprocessed/spreadsheet.json
Synopses: ../../data/unprocessed/synopses/
Studies: ../../data/unproce

In [2]:
!python train.py \
    --model CuML \
    --data-path=../../data/level-0.5/data.json \
    --output-path=./models/level-1/cuML_classifier.pkl \
    --timer





Loading data...

/workspace/GreyLit/venv/lib/python3.10/site-packages/cudf/io/json.py:108: UserWarning: Using CPU via Pandas to read JSON dataset, this may be GPU accelerated in the future
  warnings.warn(
Data loaded. 
Splitting data...
Data split.

Initialising CuML trainer...

Train data:
<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 15081 entries, 0 to 15080
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   url           15081 non-null  object
 1   text          15081 non-null  object
 2   relevance     15081 non-null  object
 3   multiclasses  15081 non-null  list
dtypes: list(1), object(3)
memory usage: 1.0+ GB

Test data:
<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 3771 entries, 15081 to 18851
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   url           3771 non-null   object
 1   text          3771 non-null   object
 2   

In [3]:
!python predict.py \
    --model=CuML \
    --model-path=./models/level-1/cuML_classifier.pkl \
    --data-path=../../data/level-0.5/irrelevant.json \
    --output-path=../../results/level-1.5 \
    --level 1 \
    --save-top 200 \
    --timer




Traceback (most recent call last):
  File "/workspace/GreyLit/GreyLiteratureClassifier/src/scripts/predict.py", line 195, in <module>
    main(**vars(args))
  File "/workspace/GreyLit/GreyLiteratureClassifier/src/scripts/predict.py", line 165, in main
    test_cuML(**kwargs)
TypeError: test_cuML() got an unexpected keyword argument 'model'


In [4]:
!python predict.py \
    --model=FastFit \
    --model-path=./models/level-2/avsolatorio/GIST-Embedding-v0 \
    --data-path=../../data/level-1.5/potential.json \
    --output-path=../../results/level-2.5/ \
    --save-top 200 \
    --level 2 \
    --timer




Loading model from ./models/level-2/avsolatorio/GIST-Embedding-v0 ...
Model loaded.
Tokenizer loaded with max_length: 512
The model 'FastFit' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'F

In [5]:
!python train.py \
    --model FastFit \
    --data-path=../../data/level-0.5/data.json \
    --embedding-model=avsolatorio/GIST-Embedding-v0 \
    --output-path=./models/level-2/ \
    --chunk-size 512 \
    --batch-size 32 \
    --samples-per-label 100000 \
    --timer





Loading data...
Data loaded. 
Splitting data...
Data split.

Chunking data...
100%|█████████████████████████████████████| 3017/3017 [00:01<00:00, 2932.06it/s]
Data chunked.

Downsampling data...
Data downsampled.

Converting data to FastFit format...

Train data:
Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses'],
    num_rows: 200000
})

Test data:
Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses', '__index_level_0__'],
    num_rows: 200
})

Initialising FastFit trainer...
08/20/2024 12:13:37 - WARNING - fastfit.train - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True
Running tokenizer on dataset to infer max length for both query and document: 10
Running tokenizer on dataset to infer max length for both query and document: 10
Running tokenizer on dataset to infer max length for both query and document: 10
Running tokenizer on dataset: 100%|█| 200000/200000 [02:57<00:00, 1127.44 